Combine photo-label map and photo-restaurant map

In [1]:
import os
import re
import time
import random
from glob import glob
import pandas as pd

In [2]:
path = "/data/trainnew"
image_filenames = glob('{}/*.jpg'.format(path))

In [6]:
int(image_filenames[0].split("/")[-1].split('.jpg')[0])

10

In [7]:
train_list = [ int(x.split("/")[-1].split('.jpg')[0]) for x in image_filenames]

In [10]:
train_list.sort()
train_list[0:5]

[5, 10, 14, 20, 24]

In [22]:
train_pic_biz = pd.read_csv('/data/train_photo_to_biz_ids.csv')
train_biz_label = pd.read_csv('/data/train.csv')
train_biz_label = train_biz_label.dropna()
print train_pic_biz[0:5]

   photo_id  business_id
0    204149         3034
1     52779         2805
2    278973          485
3    195284          485
4     19992          485


In [24]:
photo_id_list_df = pd.DataFrame(train_list, columns=["photo_id"])
print bus_id_list_df[0:5]

   photo_id
0         5
1        10
2        14
3        20
4        24


In [30]:
train_biz_labelnew = pd.merge(photo_id_list_df, train_pic_biz, on='photo_id', how='inner')
train_biz_labelnew[0:5]
pic_label = pd.merge(train_biz_labelnew, train_biz_label, on='business_id', how='left')

In [31]:
pic_label= pic_label.dropna()

In [32]:
pic_label[0:5]

,photo_id,business_id,labels
0,5,3305,3 6
1,10,242,0 2 5 6 8
2,14,1202,1 2 4 5 6 7 8
3,20,3799,0 2 3 5 6 7
4,24,276,0 5 6 8


In [34]:
pic_label = pic_label.sort_values(['photo_id'])
len(pic_label)

61718

In [35]:
def get_label(labels, level):
    y_label_1 = [0]*len(labels)
    for i in range(len(labels)):
        try:
            y_label_1[i] = int(level in labels[i])
        except:
            pass
    return y_label_1

In [36]:
y_label_0 = get_label(pic_label['labels'].tolist(), '0')
y_label_1 = get_label(pic_label['labels'].tolist(), '1')
y_label_2 = get_label(pic_label['labels'].tolist(), '2')
y_label_3 = get_label(pic_label['labels'].tolist(), '3')
y_label_4 = get_label(pic_label['labels'].tolist(), '4')
y_label_5 = get_label(pic_label['labels'].tolist(), '5')
y_label_6 = get_label(pic_label['labels'].tolist(), '6')
y_label_7 = get_label(pic_label['labels'].tolist(), '7')
y_label_8 = get_label(pic_label['labels'].tolist(), '8')

label_list = pd.DataFrame({'label_0' : y_label_0,'label_1' : y_label_1,'label_2' : y_label_2,'label_3' : y_label_3,
                                'label_4' : y_label_4,'label_5' : y_label_5,'label_6' : y_label_6,'label_7' : y_label_7,
                                'label_8' : y_label_8})
print label_list[0:5]

   label_0  label_1  label_2  label_3  label_4  label_5  label_6  label_7  \
0        0        0        0        1        0        0        1        0   
1        1        0        1        0        0        1        1        0   
2        0        1        1        0        1        1        1        1   
3        1        0        1        1        0        1        1        1   
4        1        0        0        0        0        1        1        0   

   label_8  
0        0  
1        1  
2        1  
3        0  
4        1  


In [37]:
pic_label.index = range(len(pic_label))

In [38]:
result =  pic_label.join(label_list)

In [39]:
result[0:5]

,photo_id,business_id,labels,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8
0,5,3305,3 6,0,0,0,1,0,0,1,0,0
1,10,242,0 2 5 6 8,1,0,1,0,0,1,1,0,1
2,14,1202,1 2 4 5 6 7 8,0,1,1,0,1,1,1,1,1
3,20,3799,0 2 3 5 6 7,1,0,1,1,0,1,1,1,0
4,24,276,0 5 6 8,1,0,0,0,0,1,1,0,1


In [41]:
result.drop(['labels'],inplace=True,axis=1)

In [42]:
result.to_csv('pic_label_subbybus_train.csv',index=False)